In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import watermark

%load_ext watermark
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.9.0
IPython version      : 7.19.0

Compiler    : Clang 12.0.0 (clang-1200.0.32.27)
OS          : Darwin
Release     : 20.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit

Git hash: 

watermark: 2.1.0
scipy    : 1.5.4
pandas   : 1.1.4
numpy    : 1.19.4



To fully support reproduction of the results based on the information that is contained in our published data set, this notebook contains all calculations that are performed for the information that is displayed in the text of the manuscript. For the readers convenience, we have grouped this information per section of the manuscript.

# Results

## Within-subject CDS prevalence

Load demographic thresholds used to infer demographics based on M3 output.

In [2]:
from util import threshold, gender_thr, age_thr

Define a function that outputs all results of Welch's unequal variances t-test. The statistic and p-value are calculated using `scipy.stats.ttest_ind`. The degrees of freedom (df) is calculated using

$$ df = \frac{\left(\frac{s_D^2}{n_D}+\frac{s_R^2}{n_R}\right)^2}{\frac{\left(\frac{s_D^2}{n_D}\right)^2}{n_D-1}+\frac{\left(\frac{s_R^2}{n_R}\right)^2}{n_R-1}} $$

in which $s^2_D$ and $s^2_R$ are the variances of the input arrays $D$ and $R$, respectively. Moreover, $n_D$ and $n_R$ are the number of observations in the input arrays $D$ and $R$.

The effect size is calculated using Cohen's $d$, which is defined as $d=\frac{\mu_D - \mu_R}{s}$, in which $\mu_D$ and $\mu_R$ denote the mean of the input arrays $D$ and $R$, respectively, and

$$ s = \sqrt{\frac{\left(n_D-1\right)\cdot s_D^2 + \left(n_R-1\right)\cdot s_R^2}{n_D+n_R-2}}$$

In [3]:
def annotate_stats(D, R, lab):
    W = stats.ttest_ind(D, R, equal_var=False)
    
    p1 = (D.var() / D.size + R.var() / R.size) ** 2
    p2 = (D.var() / D.size) ** 2 / (D.size - 1) + (R.var() / R.size) ** 2 / (R.size - 1)
    df = p1 / p2
    
    s = np.sqrt(((D.size - 1) * D.var() + (R.size - 1) * R.var()) / (D.size + R.size - 2))
    Cohens_d = (D.mean() - R.mean()) / s
    
    pstr = "P < 0.001" if W.pvalue < 0.001 else "P = {.3f}".format(W.pvalue)
    
    print("{}: $t={:.2f}$ $($df$={:.0f}); {};$ Cohen's $d={:.2f}$".format(lab, W.statistic, df, pstr, Cohens_d))

Select groups of individuals for the $D$ cohort based on demographics information as obtained from M3.

In [4]:
demo_D = pd.read_csv("data/D_demographics.tsv", sep="\t", index_col=[0], header=[0, 1])

non_org_D = demo_D[("org", "non-org")] >= threshold

all_D = demo_D[non_org_D].index
men_D = demo_D[non_org_D & (demo_D[("gender", "male")] >= gender_thr)].index
women_D = demo_D[non_org_D & (demo_D[("gender", "female")] >= gender_thr)].index
teens_D = demo_D[non_org_D & (demo_D[("age", "<=18")] >= age_thr)].index
twenties_D = demo_D[non_org_D & (demo_D[("age", "19-29")] >= age_thr)].index
thirties_D = demo_D[non_org_D & (demo_D[("age", "30-39")] >= age_thr)].index
adults_D = demo_D[non_org_D & (demo_D[("age", ">=40")] >= age_thr)].index

Select groups of individuals for the $R$ cohort based on demographics information as obtained from M3.

In [5]:
demo_R = pd.read_csv("data/R_demographics.tsv", sep="\t", index_col=[0], header=[0, 1])

non_org_R = demo_R[("org", "non-org")] >= threshold

all_R = demo_R[non_org_R].index
men_R = demo_R[non_org_R & (demo_R[("gender", "male")] >= gender_thr)].index
women_R = demo_R[non_org_R & (demo_R[("gender", "female")] >= gender_thr)].index
teens_R = demo_R[non_org_R & (demo_R[("age", "<=18")] >= age_thr)].index
twenties_R = demo_R[non_org_R & (demo_R[("age", "19-29")] >= age_thr)].index
thirties_R = demo_R[non_org_R & (demo_R[("age", "30-39")] >= age_thr)].index
adults_R = demo_R[non_org_R & (demo_R[("age", ">=40")] >= age_thr)].index

Load within-user CDS prevalence per user for both cohorts.

In [6]:
CDS_D = pd.read_csv("results/D_within_user_prevalence.tsv", sep="\t", index_col=[0])
CDS_R = pd.read_csv("results/R_within_user_prevalence.tsv", sep="\t", index_col=[0])

Calculate mean of within-user CDS prevalence for the $D$ cohort ($\bar{P}_D$).

In [7]:
CDS_D.loc[all_D, "prevalence"].mean()

0.23203790787320067

Calculate mean of within-user CDS prevalence for the $R$ cohort ($\bar{P}_R$).

In [8]:
CDS_R.loc[all_R, "prevalence"].mean()

0.17255157469621327

Determine the difference in means using Welch's unequal variances t-test for all individuals.

In [9]:
annotate_stats(CDS_D.loc[all_D, "prevalence"], CDS_R.loc[all_R, "prevalence"], r"All individuals")

All individuals: $t=21.20$ $($df$=1619); P < 0.001;$ Cohen's $d=0.56$


Determine the percentage of individuals in the $R$ cohort that have **no** tweets with CDS.

In [10]:
(CDS_R.loc[all_R, "prevalence"] == 0).sum() / all_R.size * 100

9.756429446183153

Determine the number of individuals in the $D$ ochort that have **no** tweets with CDS.

In [11]:
(CDS_D.loc[all_D, "prevalence"] == 0).sum()

4

Determine the percentage of individuals in the $D$ ochort that have **no** tweets with CDS.

In [12]:
(CDS_D.loc[all_D, "prevalence"] == 0).sum() / all_D.size * 100

0.3864734299516908

Determine the difference in means using Welch's unequal variances t-test for all demographic subgroups.

In [13]:
annotate_stats(CDS_D.loc[men_D, "prevalence"], CDS_R.loc[men_R, "prevalence"], r"Male")

Male: $t=9.82$ $($df$=335); P < 0.001;$ Cohen's $d=0.53$


In [14]:
annotate_stats(CDS_D.loc[women_D, "prevalence"], CDS_R.loc[women_R, "prevalence"], r"Female")

Female: $t=16.81$ $($df$=1127); P < 0.001;$ Cohen's $d=0.62$


In [15]:
annotate_stats(CDS_D.loc[teens_D, "prevalence"], CDS_R.loc[teens_R, "prevalence"], r"Aged 18 and under")

Aged 18 and under: $t=9.35$ $($df$=208); P < 0.001;$ Cohen's $d=0.71$


In [16]:
annotate_stats(CDS_D.loc[twenties_D, "prevalence"], CDS_R.loc[twenties_R, "prevalence"], r"Aged 19-29")

Aged 19-29: $t=13.49$ $($df$=580); P < 0.001;$ Cohen's $d=0.67$


In [17]:
annotate_stats(CDS_D.loc[thirties_D, "prevalence"], CDS_R.loc[thirties_R, "prevalence"], r"Aged 30-39")

Aged 30-39: $t=7.73$ $($df$=217); P < 0.001;$ Cohen's $d=0.59$


In [18]:
annotate_stats(CDS_D.loc[adults_D, "prevalence"], CDS_R.loc[adults_R, "prevalence"], r"Aged 40 and over")

Aged 40 and over: $t=3.49$ $($df$=103); P < 0.001;$ Cohen's $d=0.30$


## Between-cohort CDS prevalence

Load number of tweets included in each bootstrap run.

In [19]:
tweets_per_run = pd.read_csv("bootstrap/tweets_per_run.tsv", sep="\t", index_col=[0])

Calculate 95% CI of the number of tweets included in the bootstrap for the $D$ cohort.

In [20]:
tweets_per_run["$D$"].quantile(q=0.025), tweets_per_run["$D$"].quantile(q=0.975)

(1454068.75, 1566230.325)

Calculate 95% CI of the number of tweets included in the bootstrap for the $R$ cohort.

In [21]:
tweets_per_run["$R$"].quantile(q=0.025), tweets_per_run["$R$"].quantile(q=0.975)

(6630441.375, 6941408.2)

Load prevalence ratios ($PR$) of all bootstrap iterations (All individuals: `relative_prevalence.tsv`, Aged 19-29: `relative_prevalence_twenties.tsv` and Aged 40 and over: `relative_prevalence_adults.tsv`)

In [22]:
rel_prev = pd.read_csv("bootstrap/relative_prevalence.tsv", sep="\t", index_col=[0], header=None)[1]
twenties = pd.read_csv("bootstrap/relative_prevalence_twenties.tsv", sep="\t", index_col=[0], header=None)[1]
adults = pd.read_csv("bootstrap/relative_prevalence_adults.tsv", sep="\t", index_col=[0], header=None)[1]

Determine the 95% CI of the $PR$ for all individuals

In [23]:
rel_prev.quantile(q=0.025), rel_prev.quantile(q=0.975)

(1.1019263973960869, 1.15714518156635)

Determine the median value of the $PR$ for all individuals

In [24]:
rel_prev.median()

1.1292314690622147

Determine the median value of the $PR$ for all the demographic subgroups 'Aged 19-29' and 'Aged 40 and over', as these subgroups have the extreme values (observed from Figure 2).

In [25]:
adults.median(), twenties.median()

(1.1022022065176276, 1.163640415363524)

## CDS prevalence by CD type

Load cognitive distortion schemata (CDS)

In [26]:
_CDS = pd.read_csv("data/list_of_CDS.tsv", sep="\t", index_col="markers")

Load relative prevalence per category

In [27]:
RP_phrase = pd.read_csv("bootstrap/relative_prevalence_phrase.tsv", sep="\t", index_col=[0])

Sort mean relative prevalence on median values

In [28]:
sorted_vals = RP_phrase.median().sort_values().index

/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)


Determine schemata that are used significantly more in one of the two cohorts

In [29]:
CI_low = RP_phrase.loc[:, sorted_vals].quantile(q=0.025)
CI_high = RP_phrase.loc[:, sorted_vals].quantile(q=0.975)
signif_vals = sorted_vals[(CI_low > 1) | (CI_high < 1)]

/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:3968: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


Determine top schemata that are used significantly used more in the $D$ cohort

In [30]:
signif_vals[-10:]

Index(['if it only', 'because my', 'because I feel', 'all my fault',
       'a burden', 'because of my', 'I caused', 'will go wrong',
       'everyone will think', 'they will not think'],
      dtype='object')

Determine top schemata that are used significantly used more in the $R$ cohort

In [31]:
signif_vals[:10]

Index(['she will not believe', 'we will not think', 'nobody will believe',
       'she will not think', 'all my doing', 'OK yet', 'we will not believe',
       'only the worst', 'will be terrible', 'completely bad'],
      dtype='object')

# Robustness

## Absence of sentiment effect

Load all CDS and calculate their VADER sentiment score.

In [32]:
_CDS = pd.read_csv("data/list_of_CDS.tsv", sep="\t", index_col="markers")
VADER = SentimentIntensityAnalyzer()
_CDS["VADER"] = _CDS.apply(lambda x: VADER.polarity_scores(x.name)['compound'], axis=1)

Determine the percentage of CDS that has no sentiment loading (i.e., has obtained a score of `0`).

In [33]:
_CDS["VADER"][_CDS["VADER"] == 0].size / _CDS["VADER"].size * 100

75.93360995850622

Determine the average sentiment loading of all CDS.

In [34]:
_CDS["VADER"].mean()

-0.050950622406639